---
title: "xr.apply_ufunc(...)"
categories: [xarray]
date: 2025-04-11
---


## Example on how `xr.apply_ufunc(...)` can be used for pixel wise prediction

Note: `ds.map_blocks()` likely is faster compared to this approach. This is really only for demonstrative purpose, as a template to be adapted for different things.

In [1]:
import dask.array as da
import xarray as xr
import numpy as np
from sklearn.ensemble import RandomForestClassifier as RF

In [2]:
n_features = 12

In [3]:
n_samples = 1000
# random training data
X_train = da.random.random((n_samples,n_features))
y_train = da.random.randint(0, 2, n_samples)

rf = RF(random_state = 42, n_estimators=50, n_jobs=-1)
rf.fit(X_train, y_train)

RandomForestClassifier(n_estimators=50, n_jobs=-1, random_state=42)

In [4]:
# random training data
# keep size reasonably small
lat = np.arange(40)
lon = np.arange(60)
time = np.arange(n_features)
data = da.random.random((lat.size,lon.size, time.size))

In [5]:
ds = xr.DataArray(
    data,
    coords=[lat, lon, time],
    dims=["lat", "lon", "time"],
    name="test",
).to_dataset()

In [6]:
ds

<xarray.Dataset> Size: 231kB
Dimensions:  (lat: 40, lon: 60, time: 12)
Coordinates:
  * lat      (lat) int64 320B 0 1 2 3 4 5 6 7 8 9 ... 31 32 33 34 35 36 37 38 39
  * lon      (lon) int64 480B 0 1 2 3 4 5 6 7 8 9 ... 51 52 53 54 55 56 57 58 59
  * time     (time) int64 96B 0 1 2 3 4 5 6 7 8 9 10 11
Data variables:
    test     (lat, lon, time) float64 230kB dask.array<chunksize=(40, 60, 12), meta=np.ndarray>

In [7]:
def generic_func(arr):
    return rf.predict(arr.reshape(1, -1))

In [8]:
ds_ag = xr.apply_ufunc(
    generic_func,
    ds,
    input_core_dims=[["time"]],
    dask="parallelized",
    output_dtypes=np.float32,
    vectorize=True,
    dask_gufunc_kwargs={"allow_rechunk": True},
)

In [9]:
ds_ag

<xarray.Dataset> Size: 10kB
Dimensions:  (lat: 40, lon: 60)
Coordinates:
  * lat      (lat) int64 320B 0 1 2 3 4 5 6 7 8 9 ... 31 32 33 34 35 36 37 38 39
  * lon      (lon) int64 480B 0 1 2 3 4 5 6 7 8 9 ... 51 52 53 54 55 56 57 58 59
Data variables:
    test     (lat, lon) float32 10kB dask.array<chunksize=(40, 60), meta=np.ndarray>

In [10]:
ds_ag.compute()

<xarray.Dataset> Size: 10kB
Dimensions:  (lat: 40, lon: 60)
Coordinates:
  * lat      (lat) int64 320B 0 1 2 3 4 5 6 7 8 9 ... 31 32 33 34 35 36 37 38 39
  * lon      (lon) int64 480B 0 1 2 3 4 5 6 7 8 9 ... 51 52 53 54 55 56 57 58 59
Data variables:
    test     (lat, lon) float32 10kB 1.0 1.0 0.0 0.0 1.0 ... 1.0 0.0 0.0 0.0 0.0